## Code To Work and Test with Smolsagent

In [ ]:
import litellm
import os
from dotenv import load_dotenv

load_dotenv()

response = litellm.completion(
    model="github/gpt-4.1-nano",
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
    ],
    api_key=os.getenv("GITHUB_API_KEY"),
)

In [14]:
from typing import Any
from smolagents import CodeAgent, tool, LiteLLMModel, FinalAnswerTool
import os
from dotenv import load_dotenv

load_dotenv()

SYSTEM_PROMPT = (
    "You are a knowledgeable assistant in the field of flowers. 🌷 "
    "If the user asks about a flower, you should provide information about it. "
    "However, if the user asks about a flower that is not in your knowledge, "
    "you should say 'I don't know about that flower. 🤷‍♀️'. "
    "You should ONLY answer about flowers. If the question is not about flowers, "
    "politely state that you only discuss flowers. "
    "ALWAYS use flower emojis in your answers (e.g., 🌹, 🌻, 🌸, 🌼, 🌷)."
    "MUST KEEP YOUR RESPONSES SHORT AND CONCISE. "
)

model = LiteLLMModel(
    model_id="github/gpt-4.1-nano",
    api_key=os.getenv("GITHUB_API_KEY"),
    temperature=0.5,
)

@tool
def get_flower_info(flower_name: str) -> str:
    """
    Get information about a flower. 
    If there is non-related content to flower or you don't know, then say "I don't know about that flower. 🤷‍♀️"
    "
    Args:
        flower_name (str): The name of the flower.
    Returns:
        str: Information about the flower.
    """
    if flower_name.lower() in ["rose", "tulip", "daisy"]:
        return f"The {flower_name} is a beautiful flower! 🌹🌷🌼"
    elif flower_name.lower() in ["sunflower", "lily", "orchid"]:
        return f"The {flower_name} is a stunning flower! 🌻🌸🌺"
    else:
        return "I don't know about that flower. 🤷‍♀️"
    
from smolagents.memory import MemoryStep

def print_step(step: MemoryStep) -> None:
    print("Steps taken:")
    print(step.model_output)

agent = CodeAgent(
    model=model,
    tools=[
        get_flower_info,
    ],
    max_steps=3,
    step_callbacks=[print_step],
    verbosity_level=0,
)

response = agent.run("Tell me something about rose.")
print(response)

Steps taken:
Thought: I will gather some general information about roses to fulfill the task. Since I only have access to the `get_flower_info` tool, I will use it to get details specifically about roses.
Code:
```py
info = get_flower_info("rose")
final_answer(info)
```<end_code>
The rose is a beautiful flower! 🌹🌷🌼


In [12]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from smolagents import Tool
from langchain_community.retrievers import BM25Retriever
from smolagents import CodeAgent, InferenceClientModel, LiteLLMModel
import os
from dotenv import load_dotenv

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

if GEMINI_API_KEY is None:
    raise ValueError("GEMINI_API_KEY environment variable not set.")

MODEL_ID = "gemini/gemini-2.0-flash"  # Replace with the actual model ID you want to use

gemini_litellm = LiteLLMModel(
    model_id=MODEL_ID,
    api_key=GEMINI_API_KEY,
    temperature=0.7,
)

class PartyPlanningRetrieverTool(Tool):
    name = "party_planning_retriever"
    description = "Uses semantic search to retrieve relevant party planning ideas for Alfred’s superhero-themed party at Wayne Manor."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be a query related to party planning or superhero themes.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        self.retriever = BM25Retriever.from_documents(
            docs, k=5  # Retrieve the top 5 documents
        )

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.retriever.invoke(
            query,
        )
        return "\nRetrieved ideas:\n" + "".join(
            [
                f"\n\n===== Idea {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

# Simulate a knowledge base about party planning
party_ideas = [
    {"text": "A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.", "source": "Party Ideas 1"},
    {"text": "Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.", "source": "Entertainment Ideas"},
    {"text": "For catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'", "source": "Catering Ideas"},
    {"text": "Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.", "source": "Decoration Ideas"},
    {"text": "Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.", "source": "Entertainment Ideas"}
]

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in party_ideas
]

# Split the documents into smaller chunks for more efficient search
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
docs_processed = text_splitter.split_documents(source_docs)

# Create the retriever tool
party_planning_retriever = PartyPlanningRetrieverTool(docs_processed)

# Initialize the agent
agent = CodeAgent(tools=[party_planning_retriever], model=gemini_litellm, step_callbacks=[print_steps], verbosity_level=0)

# Example usage
response = agent.run(
    "Find ideas for a luxury superhero-themed party, including entertainment, catering, and decoration options."
)

print(response)

Steps taken:
ActionStep(model_input_messages=[{'role': <MessageRole.SYSTEM: 'system'>, 'content': [{'type': 'text', 'text': 'You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \'Thought:\', \'Code:\', and \'Observation:\' sequences.\n\nAt each step, in the \'Thought:\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\nThen in the \'Code:\' sequence, you should write the code in simple Python. The code sequence must end with \'<end_code>\' sequence.\nDuring each intermediate step, you can use \'print()\' to save whatever important information you will then need.\nThese print outputs will then appear in the \'Observation:\' field, which will b

In [5]:
agent.run(
    "Suggest superheros who should be invited to Alfred's party at Wayne Manor. "
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Suggest superheros who should be invited to Alfred's party at Wayne Manor.                                      │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash ────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  superhero_suggestions = party_planning_retriever(query="superhero guests for Wayne Manor party")                 
  final_answer(superhero_suggestions)                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 
Retrieved ideas:


===== Idea 0 =====
Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.

===== Idea 1 =====
Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.

===== Idea 2 =====
Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.

===== Idea 3 =====
For catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'

===== Idea 4 =====
A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.

[Step 1: Duration 2.25 seconds| Input tokens: 2,236 | Output tokens: 84]

"\nRetrieved ideas:\n\n\n===== Idea 0 =====\nInteractive experiences with VR where guests can engage in superhero simulations or compete in themed games.\n\n===== Idea 1 =====\nHire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.\n\n===== Idea 2 =====\nDecorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.\n\n===== Idea 3 =====\nFor catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'\n\n===== Idea 4 =====\nA superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains."

In [ ]:
print(agent.memory.steps[1].dict()['model_output'])

Thought: I need to find ideas for a luxury superhero-themed party. This includes entertainment, catering, and decoration options. I will use the `party_planning_retriever` tool to get relevant party planning ideas. I will start with a broad query and then refine it based on the results.
Code:
```py
query = "luxury superhero themed party ideas entertainment catering decoration"
party_ideas = party_planning_retriever(query=query)
print(party_ideas)
```<end_code>


In [ ]:
agent.run()

The 'logs' attribute is deprecated and will soon be removed. Please use 'self.memory.steps' instead.


[SystemPromptStep(system_prompt='You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \'Thought:\', \'Code:\', and \'Observation:\' sequences.\n\nAt each step, in the \'Thought:\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\nThen in the \'Code:\' sequence, you should write the code in simple Python. The code sequence must end with \'<end_code>\' sequence.\nDuring each intermediate step, you can use \'print()\' to save whatever important information you will then need.\nThese print outputs will then appear in the \'Observation:\' field, which will be available as input for the next step.\nIn the end you have to return a final answer using 